In [6]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

You should consider upgrading via the '/Users/krzysztofpraca/Projects/hackathon-2023/venv/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 60 kB 1.9 MB/s eta 0:00:011
     |████████████████████████████████| 50 kB 9.4 MB/s  eta 0:00:01
You should consider upgrading via the '/Users/krzysztofpraca/Projects/hackathon-2023/venv/bin/python3 -m pip install --upgrade pip' command.


In [28]:
!pip install tqdm

     |████████████████████████████████| 78 kB 2.7 MB/s eta 0:00:01
You should consider upgrading via the '/Users/krzysztofpraca/Projects/hackathon-2023/venv/bin/python3 -m pip install --upgrade pip' command.


In [39]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

from tqdm import tqdm

In [40]:
df = pd.read_csv('worldcities.csv')

In [41]:
ethiopia_df = df[df.country == 'Ethiopia']

In [42]:
ethiopia_df[:20]

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
259,Addis Ababa,Addis Ababa,9.0300,38.7400,Ethiopia,ET,ETH,Ādīs Ābeba,primary,3041002.0,1231824991
923,Godē,Gode,5.9527,43.5516,Ethiopia,ET,ETH,Sumalē,NaN,980782.0,1231128743
1267,Ērer Sātā,Erer Sata,9.5667,41.3833,Ethiopia,ET,ETH,Sumalē,NaN,649000.0,1231563638
2130,Nazrēt,Nazret,8.5414,39.2689,Ethiopia,ET,ETH,Oromīya,NaN,324000.0,1231826873
2186,Gonder,Gonder,12.6075,37.4592,Ethiopia,ET,ETH,Āmara,NaN,323900.0,1231234833
2188,Mekele,Mekele,13.4969,39.4769,Ethiopia,ET,ETH,Tigray,admin,323700.0,1231468531
2301,Āwasa,Awasa,7.0500,38.4667,Ethiopia,ET,ETH,YeDebub Bihēroch Bihēreseboch na Hizboch,admin,300100.0,1231663122
2490,Dire Dawa,Dire Dawa,9.6000,41.8667,Ethiopia,ET,ETH,Dirē Dawa,admin,277000.0,1231089714
2768,Bahir Dar,Bahir Dar,11.6000,37.3833,Ethiopia,ET,ETH,Āmara,admin,243300.0,1231784118
3048,Shashemenē,Shashemene,7.2000,38.6000,Ethiopia,ET,ETH,Oromīya,NaN,208368.0,1231110370


In [43]:
from datetime import datetime

date_tuples = [
    (datetime(year, 5, 1).strftime('%Y-%m-%d'), datetime(year, 5, 31).strftime('%Y-%m-%d'))
    for year in range(2023, 2014, -1)
]

date_tuples[::-1]

[('2015-05-01', '2015-05-31'),
 ('2016-05-01', '2016-05-31'),
 ('2017-05-01', '2017-05-31'),
 ('2018-05-01', '2018-05-31'),
 ('2019-05-01', '2019-05-31'),
 ('2020-05-01', '2020-05-31'),
 ('2021-05-01', '2021-05-31'),
 ('2022-05-01', '2022-05-31'),
 ('2023-05-01', '2023-05-31')]

In [44]:
def get_temp_time_period(lat, lng, time_start, time_end):
    url = "https://archive-api.open-meteo.com/v1/archive"
    
    params = {
    	"latitude": lat,
    	"longitude": lng,
    	"start_date": time_start,
    	"end_date": time_end,
    	"hourly": "temperature_2m"
    }
    
    responses = openmeteo.weather_api(url, params=params)

    hourly = responses[0].Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    
    hourly_data = {"date": pd.date_range(
    	start = pd.to_datetime(hourly.Time(), unit = "s"),
    	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
    	freq = pd.Timedelta(seconds = hourly.Interval()),
    	inclusive = "left"
    )}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    df_res = pd.DataFrame(hourly_data)['temperature_2m']
    
    return {
        'mean_temp': df_res.mean(),
        'std_tmp': df_res.std(),
        'min_temp': df_res.min(),
        'max_tmp': df_res.max()
    }

In [45]:
get_temp_time_period(lat=-1.2864, lng=36.8172, 
                     time_start='2023-08-01', time_end='2023-08-31')

{'mean_temp': 18.589441,
 'std_tmp': 3.764236,
 'min_temp': 11.3885,
 'max_tmp': 27.4385}

In [46]:
out_df_rows = []

for idx, row in tqdm(ethiopia_df[:20].iterrows()):
    for time_range in date_tuples:
        temp_results = get_temp_time_period(lat=row['lat'],
                             lng=row['lng'],
                             time_start=time_range[0],
                             time_end=time_range[1])

        df_row = {
            'city': row['city'],
            'date_start': time_range[0],
            'date_end': time_range[1]
        }

        df_row.update(temp_results)
        
        out_df_rows.append(df_row)

20it [00:13,  1.45it/s]


In [47]:
pd.DataFrame(out_df_rows).to_csv('temperature.csv', index=False)

In [26]:
ethiopia_df

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
259,Addis Ababa,Addis Ababa,9.0300,38.7400,Ethiopia,ET,ETH,Ādīs Ābeba,primary,3041002.0,1231824991
923,Godē,Gode,5.9527,43.5516,Ethiopia,ET,ETH,Sumalē,NaN,980782.0,1231128743
1267,Ērer Sātā,Erer Sata,9.5667,41.3833,Ethiopia,ET,ETH,Sumalē,NaN,649000.0,1231563638
2130,Nazrēt,Nazret,8.5414,39.2689,Ethiopia,ET,ETH,Oromīya,NaN,324000.0,1231826873
2186,Gonder,Gonder,12.6075,37.4592,Ethiopia,ET,ETH,Āmara,NaN,323900.0,1231234833
...,...,...,...,...,...,...,...,...,...,...,...
42305,Ādamī Tulu,Adami Tulu,7.8667,38.7000,Ethiopia,ET,ETH,Oromīya,NaN,9034.0,1231102446
44176,Shendē,Shende,10.6333,36.9333,Ethiopia,ET,ETH,Āmara,NaN,9293.0,1231251281
44181,Mēga,Mega,4.0167,38.2500,Ethiopia,ET,ETH,Oromīya,NaN,9370.0,1231972849
44216,Bīkē,Bike,9.5297,41.1969,Ethiopia,ET,ETH,Sumalē,NaN,9197.0,1231520658
